In [48]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import time

In [49]:
def draw_landmarks(detection_results, image):

    width = image.shape[1]
    height = image.shape[0]

    for hand_landmarks in detection_results.multi_hand_landmarks:
        for landmark in hand_landmarks.landmark:
            x = min(int(landmark.x * width), width - 1)
            y = min(int(landmark.y * height), height - 1)
            cv.circle(image, (x, y), 5, (0, 255, 0), -1)

    return image

In [50]:
def find_center_of_hand(hand_landmarks):
    x_vals = [landmark.x for landmark in hand_landmarks.landmark]
    y_vals = [landmark.y for landmark in hand_landmarks.landmark]
    z_vals = [landmark.z for landmark in hand_landmarks.landmark]

    x_center = sum(x_vals) / len(x_vals)
    y_center = sum(y_vals) / len(y_vals)
    z_center = sum(z_vals) / len(z_vals)

    return x_center, y_center, z_center

In [52]:
def flatten_scale_relative(results):

    flattened_scaled_results = []
    for hand_landmarks in results.multi_hand_landmarks:
        x_wrist = hand_landmarks.landmark[0].x
        y_wrist = hand_landmarks.landmark[0].y

        for landmark in hand_landmarks.landmark:
            flattened_scaled_results.append(landmark.x - x_wrist)
            flattened_scaled_results.append(landmark.y - y_wrist)
            flattened_scaled_results.append(landmark.z)
    
    return flattened_scaled_results

In [53]:
def write_to_csv(results, csv_path, category):
    csv_row = [category]
    csv_row.extend(flatten_scale_relative(results))

    # write to csv
    with open(csv_path, 'a') as f:
        f.write(','.join([str(x) for x in csv_row]) + '\n')


In [11]:
# hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7,
#                                  min_tracking_confidence=0.5)

# img = cv.imread('Data\istockphoto-1267273871-612x612.jpg')
# img = cv.flip(img, 1)


# cv.imshow('image', img)

# results = hands.process(cv.cvtColor(img, cv.COLOR_BGR2RGB))

# write_to_csv(results, 'Data\hands.csv', 0)

# cv.waitKey(0)
# cv.destroyAllWindows()

In [54]:
def log_data(csv_path, category):
    cap = cv.VideoCapture(0)
    hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7,
                                    min_tracking_confidence=0.5)


    time.sleep(7)
    print('STARTING')
    while True:
        success, img = cap.read()
        img = cv.flip(img, 1)

        if not success:
            print("Could not read frame")
            break

        results = hands.process(cv.cvtColor(img, cv.COLOR_BGR2RGB))

        if results.multi_hand_landmarks:
            
            if len(results.multi_hand_landmarks) == 2:
                write_to_csv(results, csv_path, category)
                            
            if len(results.multi_hand_landmarks) < 2:
                print('NOT WRITING')
            
            img = draw_landmarks(results, img)
        
        cv.imshow("image", img)

        if cv.waitKey(1) == 27:  # Press ESC to exit
            break

    cap.release()
    cv.destroyAllWindows()

In [55]:
# collecting data for steering position
log_data('Data\steering.csv', 0)

STARTING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
NOT WRITING
